In [1]:
%matplotlib inline
import pymongo
from datetime import datetime
from stop_words import get_stop_words
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
s=get_stop_words('english')+[" ",'']
pd.set_option("display.max_rows",30)

In [2]:
connection=pymongo.MongoClient("mongodb://localhost")['News']['Headlines']
buzzfeed=pd.DataFrame(list(connection.find({'site':'buzzfeed'}).limit(500)))
nytimes=pd.DataFrame(list(connection.find({'site':'ny_times'}).skip(600).limit(500)))

In [3]:
buzzfeed.drop('_id', axis=1, inplace=True)
nytimes.drop('_id', axis=1, inplace=True)


In [4]:
print(len(buzzfeed[buzzfeed.title.str.match('\d')]))
print(len(nytimes[nytimes.title.str.match('\d')])) 

192
8


In [5]:
print(len(buzzfeed[buzzfeed.title.str.contains('\d')]))
print(len(nytimes[nytimes.title.str.contains('\d')])) 

230
96


In [7]:
print(len(buzzfeed[buzzfeed.title.str.match("Here's")]))
print(len(nytimes[nytimes.title.str.match("Here's")]))#Here's, This, These, 

10
0


In [8]:
print(len(buzzfeed[buzzfeed.title.str.match("This")]))
print(len(nytimes[nytimes.title.str.match("This")]))#Here's, This, These, 

47
1


In [9]:
print(len(buzzfeed[buzzfeed.title.str.match("These")]))
print(len(nytimes[nytimes.title.str.match("These")]))#Here's, This, These, 

4
0


In [10]:
print(len(buzzfeed[buzzfeed.title.str.endswith('?')]))
print(len(nytimes[nytimes.title.str.endswith("?")]))

47
5


In [11]:
nytimes.index=nytimes.site
buzzfeed.index=buzzfeed.site
ny_titles=nytimes.ix[:,1]
buzz_titles=buzzfeed.ix[:,1]


In [12]:
ny=ny_titles.str.lower().str.replace("[^\w\s#]",'').str.split().apply(pd.Series, 1).stack()
buzz=buzz_titles.str.lower().str.replace("[^\w\s#]",'').str.split().apply(pd.Series, 1).stack()

In [13]:
ny.index= ny.index.droplevel(-1)
buzz.index=buzz.index.droplevel(-1)

In [14]:
ny=ny[~ny.isin(s)]
buzz=buzz[~buzz.isin(s)]

In [15]:
ny_count=ny.value_counts()
buzz_count=buzz.value_counts()

In [17]:
buzz

site
buzzfeed        clipon
buzzfeed           man
buzzfeed          buns
buzzfeed    absolutely
buzzfeed          must
buzzfeed       stopped
buzzfeed         first
buzzfeed       finding
buzzfeed          dory
buzzfeed       trailer
buzzfeed      suggests
buzzfeed          dory
buzzfeed          went
buzzfeed       missing
buzzfeed            21
               ...    
buzzfeed       colbert
buzzfeed         broke
buzzfeed          news
buzzfeed         paris
buzzfeed       victims
buzzfeed         paris
buzzfeed       attacks
buzzfeed        syrian
buzzfeed      refugees
buzzfeed        france
buzzfeed           say
buzzfeed         paris
buzzfeed        terror
buzzfeed        terror
buzzfeed          fled
dtype: object

In [18]:
print(buzz_count[buzz_count.index=='paris'])
print(ny_count[ny_count.index=='paris'])

paris    33
dtype: int64
paris    18
dtype: int64


In [31]:
Buzzfeed_Test=['27 People Who Have Better Job Titles Than You',
               'This Woman Is Trying To Make Fake Freckles The Next Big Beauty Trend',
               'Someone Used 500 Dick Pics To Make A Regal Portrait Of Donald Trump',
               "Here's How Much It Actually Costs To Be A Bridesmaid",
               "Chocolate Abuelita Is The Best Thing Ever And Here's Why",
               "What It's Actually Like Being Vegetarian On Thanksgiving",
               'Freddie From "ICarly" Got Hitched And Your Childhood Is Now Over',
               "17 Mouthwatering Risottos Guaranteed To Change Your Life",
               "Let's Settle This Once And For All: How Should Dogs Wear Pants?",
               "These Twin Astronauts Have The Most Epic Sibling Rivalry Ever",
              ]
NY_Test=['Chicago Wins Bid to Host Obama Library',
        "Christie Sidesteps New Jersey Bridge Scandal in ’16 Race",
        "Report Assails Baltimore Police’s Response to Unrest",
        "Times Square’s Biggest and Most Expensive Digital Billboard Is Set to Shine",
        "Obama Defends His Strategy on ISIS",
        "Democrats Question Republican-Led Inquiry Into Hillary Clinton’s Aide",
        "Why Has Trust in the News Media Declined?",
        "Taking Stock of the $10 Billion Washington Spent on Colombia’s War",
        "Slim Hopes Are Dashed as Rutgers Peters Out",
        "End of China’s One-Child Policy Stings Its ‘Loneliest Generation"]


In [26]:
def first_word_frac(test_title):
    if re.match(r'\d',test_title):
        buzz_val=len(buzzfeed[buzzfeed.title.str.match('\d')])
        ny_val=len(nytimes[nytimes.title.str.match('\d')])
    else:
        first_word=test_title.split(' ')[0]
        buzz_val=len(buzzfeed[buzzfeed.title.str.match(first_word)])
        ny_val=len(nytimes[nytimes.title.str.match(first_word)])
        if buzz_val+ny_val==0:
            return {'buzz_frac':0,'ny_frac':0}
    return {'buzz_frac':buzz_val/(ny_val+buzz_val),'ny_frac':ny_val/(ny_val+buzz_val)}
    

In [34]:
def word_prob(test_title,buzz_frac,ny_frac):
    for word in re.sub(r'[^\w\s#]','',test_title).lower().split():
        b=buzz_count[buzz_count.index==word]
        n=ny_count[ny_count.index==word]
        if not b.empty and n.empty:
            buzz_frac=(1+buzz_frac)/2
            ny_frac=ny_frac/2
        elif b.empty and not n.empty:
            buzz_frac=buzz_frac/2
            ny_frac=(1+ny_frac)/2
        elif not b.empty and not n.empty:
            buzz_frac=(b.item()/(b.item()+n.item())+buzz_frac)/2
            ny_frac=(n.item()/(b.item()+n.item())+ny_frac)/2
            
    if buzz_frac > ny_frac:
        print("Buzzfeed Title")
    elif buzz_frac < ny_frac:
        print("NY Times Title")
    else:
        print("Undecided")
    

In [35]:
for title in Buzzfeed_Test:
    dic=first_word_frac(title)
    word_prob(title,**dic)

NY Times Title
Buzzfeed Title
NY Times Title
Buzzfeed Title
Buzzfeed Title
Buzzfeed Title
Buzzfeed Title
Buzzfeed Title
NY Times Title
Buzzfeed Title


In [36]:
for title in NY_Test:
    dic=first_word_frac(title)
    word_prob(title,**dic)
    

NY Times Title
NY Times Title
Undecided
NY Times Title
NY Times Title
NY Times Title
NY Times Title
Buzzfeed Title
NY Times Title
NY Times Title
